In this notebook we explore different ways of adding noise to the circuits.

In [1]:
import numpy as np
import sympy
import cirq
import tensorflow_quantum as tfq
from tqdm import tqdm
import tensorflow as tf

from utilities.variational import VQE
from utilities.circuit_basics import Evaluator
from utilities.idinserter import IdInserter
from utilities.simplifier import Simplifier
from utilities.unitary_killer import UnitaryMurder

n_qubits = 4
qlr = 0.01
qepochs = 10**3
verbose=0
g=1
J=0
noise=0.0
problem="TFIM"
vqe_handler = VQE(n_qubits=n_qubits, lr=qlr, epochs=qepochs, patience=100,
                  random_perturbations=True, verbose=verbose, g=g, J = J, noise=noise, problem=problem)

iid = IdInserter(n_qubits=n_qubits)
Simp = Simplifier(n_qubits=n_qubits)
killer = UnitaryMurder(vqe_handler)
indexed_circuit=[vqe_handler.number_of_cnots+k for k in range(vqe_handler.n_qubits,2*vqe_handler.n_qubits)]

In [2]:
circuit, symbols, idx_symbols = vqe_handler.give_circuit(indexed_circuit) 
circuit

(0, 0): ───Rx(th_0)───

(0, 1): ───Rx(th_1)───

(0, 2): ───Rx(th_2)───

(0, 3): ───Rx(th_3)───

In [3]:
circuit.with_noise(cirq.depolarize(0.01))

(0, 0): ───Rx(th_0)───D(0.01)[<virtual>]───

(0, 1): ───Rx(th_1)───D(0.01)[<virtual>]───

(0, 2): ───Rx(th_2)───D(0.01)[<virtual>]───

(0, 3): ───Rx(th_3)───D(0.01)[<virtual>]───

In [4]:
circuit_res= cirq.resolve_parameters(circuit, {s:0 for s,k in zip(symbols, range(len(symbols)))})
tfq_circuit_noise = tfq.convert_to_tensor([circuit_res])
noisy_expectation = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.01)))
tfq_layer_noisy = noisy_expectation(tfq_circuit_noise, operators=tfq.convert_to_tensor([vqe_handler.observable]))
noisy_energy = np.squeeze(tf.math.reduce_sum(tfq_layer_noisy, axis=-1))
noisy_energy

array(-3.9466667, dtype=float32)

In [5]:
pauls = [cirq.X, cirq.Y, cirq.Z]
circuit_res = cirq.resolve_parameters(circuit, {s:0 for s,k in zip(symbols, range(len(symbols)))})
p=0.01
cbatch = []
totals=10**3
for k in range(totals):
    circ = circuit_res.copy()
    
    if k<int((1-p)*totals):
        cbatch.append(circ)
    elif int((1-p)*totals)<=k:
        which = np.random.choice(range(3), 1)[0]
        gate = pauls[which]
        for q in vqe_handler.qubits:
            circ.append(gate.on(q))
        cbatch.append(circ)

In [6]:
tfq_circuit = tfq.convert_to_tensor(cbatch)
expectation = tfq.layers.Expectation()
tfq_layer = expectation(tfq_circuit, operators=tfq.convert_to_tensor([vqe_handler.observable]*1000))
averaged_unitaries = tf.math.reduce_mean(tfq_layer, axis=0)
energy = np.squeeze(tf.math.reduce_sum(averaged_unitaries, axis=-1))
energy

array(-3.96, dtype=float32)

In [7]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(circuit_input, symbol_names=symbols, operators=tfq.convert_to_tensor([vqe_handler.observable]*1000),
        initializer=tf.keras.initializers.RandomUniform(minval=-np.pi, maxval=np.pi))

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=vqe_handler.lr)
model.compile(optimizer=adam, loss='mse')
tfqcircuit = tfq.convert_to_tensor(cbatch)
qoutput = tf.ones((10**3, 1))*vqe_handler.lower_bound_Eg
h=model.fit(x=tfqcircuit, y=qoutput, batch_size=10**3, epochs=100,
          verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=vqe_handler.patience, mode="min", min_delta=10**-3)])


Train on 1000 samples
Epoch 1/100
1000/1000 [==============================] - 1s 1ms/sample


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
energy = np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))


In [ ]:
def TFQ_model(vqe_handler, symbols, symbols_to_values=None):
    """
    symbols: continuous parameters to optimize on
    symbol_to_value: if not None, dictionary with initial seeds
    """
    circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
    output = tfq.layers.Expectation()(
                circuit_input,
                symbol_names=symbols,
                operators=tfq.convert_to_tensor([vqe_handler.observable]),
                initializer=tf.keras.initializers.RandomUniform(minval=-np.pi, maxval=np.pi))
    model = tf.keras.Model(inputs=circuit_input, outputs=output)
    adam = tf.keras.optimizers.Adam(learning_rate=vqe_handler.lr)
    model.compile(optimizer=adam, loss='mse')

    if symbols_to_values:
        model.trainable_variables[0].assign(tf.convert_to_tensor(np.array(list(symbols_to_values.values())).astype(np.float32)))
    if vqe_handler.random_perturbations:
        ### actually add noise only %10 of the times.
        if np.random.uniform()<.1:
            model.trainable_variables[0].assign( model.trainable_variables[0] + tf.random.uniform(model.trainable_variables[0].shape.as_list())*np.pi/2 )
    return model



def train_model(vqe_handler, circuit, model):
    """
    circuit: cirq object with the parametrized unitaries unresolved (sympy symbols)
    model: TFQ_model output
    """

    ## testing: if there's not parametrized unitary on every qubit, raise error.
    effective_qubits = list(circuit.all_qubits())
    for k in vqe_handler.qubits:
        if k not in effective_qubits:
            raise Error("NOT ALL QUBITS AFFECTED")

    tfqcircuit = tfq.convert_to_tensor([circuit])

    qoutput = tf.ones((1, 1))*vqe_handler.lower_bound_Eg
    h=model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=vqe_handler.epochs,
              verbose=vqe_handler.verbose, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=vqe_handler.patience, mode="min", min_delta=10**-3), TimedStopping(seconds=vqe_handler.max_time_training)])
    energy = np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))
    return energy,h
